In [51]:
# Get the trading pairs from Binance
import requests

def get_binance_currencies():
    # Binance API endpoint for exchange information
    endpoint = "https://api.binance.com/api/v3/exchangeInfo"

    # Make a GET request to the endpoint
    response = requests.get(endpoint)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        exchange_info = response.json()

        base_currency = [symbol_info['baseAsset'] for symbol_info in exchange_info['symbols']]
        quote_currency = [symbol_info['quoteAsset'] for symbol_info in exchange_info['symbols']]
        trading_pairs = [symbol_info['symbol'] for symbol_info in exchange_info['symbols']]

        return trading_pairs,base_currency,quote_currency
    else:
        print(f"Failed to retrieve currencies. Status code: {response.status_code}")
        return None
    


In [52]:
trading_pairs,base_curriencies,quote_currencies = get_binance_currencies()

In [53]:
trading_pairs[:10]

['ETHBTC',
 'LTCBTC',
 'BNBBTC',
 'NEOBTC',
 'QTUMETH',
 'EOSETH',
 'SNTETH',
 'BNTETH',
 'BCCBTC',
 'GASBTC']

In [38]:
# Create DataFrame
import pandas as pd
pairs = pd.DataFrame({
    "pair":trading_pairs,
    "base":base_curriencies,
    "quote":quote_currencies
})

In [39]:
sample = pairs[:500]

In [40]:
sample.head()

,pair,base,quote
0,ETHBTC,ETH,BTC
1,LTCBTC,LTC,BTC
2,BNBBTC,BNB,BTC
3,NEOBTC,NEO,BTC
4,QTUMETH,QTUM,ETH


In [41]:
def get_triangle():
    triangles = []
    for currency1 in sample['base'].unique():
        for currency2 in sample.loc[sample['base']==currency1,'quote'].unique():
            for currency3 in sample.loc[sample['base']==currency2,'quote'].unique():
                if sample.loc[(sample['base']==currency1) & (sample['quote']==currency3)].shape[0] > 0:
                    triangles.append([currency1+currency2, currency2+currency3, currency1+currency3])
    return triangles


In [42]:
triangles = get_triangle()

In [54]:
triangles[:10]

[['ETHBTC', 'BTCUSDT', 'ETHUSDT'],
 ['ETHBTC', 'BTCTUSD', 'ETHTUSD'],
 ['ETHBTC', 'BTCPAX', 'ETHPAX'],
 ['ETHBTC', 'BTCUSDC', 'ETHUSDC'],
 ['ETHTUSD', 'TUSDBTC', 'ETHBTC'],
 ['ETHTUSD', 'TUSDUSDT', 'ETHUSDT'],
 ['ETHPAX', 'PAXBTC', 'ETHBTC'],
 ['ETHPAX', 'PAXUSDT', 'ETHUSDT'],
 ['ETHPAX', 'PAXTUSD', 'ETHTUSD'],
 ['ETHUSDC', 'USDCUSDT', 'ETHUSDT']]

In [61]:
def check_triangular_arbitrage_opportunity(triangles):

    global df 
    highest_arbitrage = 0
    best_triangle = None
    
    for triangle in triangles:
        
        pair1 = triangle[0]
        pair2 = triangle[1]
        pair3 = triangle[2]

        # Pass null values if the pair is not in the book ticker
        if pair1  not in book_ticker_df['Symbol'].values:
            continue
        if pair2 not in book_ticker_df['Symbol'].values:
            continue
        if pair3 not in book_ticker_df['Symbol'].values:
            continue

        ask_AB = book_ticker_df.loc[book_ticker_df['Symbol']==pair1,'Ask Price'].values[0]
        ask_BC = book_ticker_df.loc[book_ticker_df['Symbol']==pair2,'Ask Price'].values[0]
        ask_AC = book_ticker_df.loc[book_ticker_df['Symbol']==pair3,'Ask Price'].values[0]

        rate_1 = ask_AC/ask_AB * ask_BC

        bid_AB = book_ticker_df.loc[book_ticker_df['Symbol']==pair1,'Bid Price'].values[0]
        bid_BC = book_ticker_df.loc[book_ticker_df['Symbol']==pair2,'Bid Price'].values[0]
        bid_AC = book_ticker_df.loc[book_ticker_df['Symbol']==pair3,'Bid Price'].values[0]

        rate_2 = bid_AB * bid_BC/bid_AC

        actual_rate = max(rate_1,rate_2)

        if (rate_1>1 or rate_2>1) and actual_rate <2:
            highest_arbitrage = max(actual_rate,highest_arbitrage)
            best_triangle = triangle
            action = "Buy" if rate_1>rate_2 else "Sell"

            if rate_1>rate_2:
                best_pair_1 = pair1
                best_pair_2 = pair2
                best_pair_3 = pair3
            else:
                best_pair_1 = pair1
                best_pair_2 = pair2
                best_pair_3 = pair3

    if highest_arbitrage > 1:
        '''print("Arbitrage Opportunity Detected!")
        print(f"Highest Arbitrage: {float(highest_arbitrage)}")
        print(f"Triangle: {best_triangle}")
        print(f"Action: {action}")'''

        # Add only distinct rows to the dataframe
        
        new_row = pd.DataFrame([[highest_arbitrage, best_pair_1,
                                 best_pair_2,best_pair_3, action]], columns=df.columns)
        df = pd.concat([df, new_row], ignore_index=True)
        df.drop_duplicates(inplace=True)
        

In [66]:
import pandas as pd
import websocket
import json
from decimal import Decimal,getcontext

getcontext().prec = 20


BINANCE_WS_BASE_URL = "wss://stream.binance.com:9443/ws/"

# Initialize an empty DataFrame
columns = ["Symbol", "Bid Price", "Ask Price"]
book_ticker_df = pd.DataFrame(columns=columns)
columns_2 = ["Highest Arbitrage", "A","B","C", "Action"]
df = pd.DataFrame(columns=columns_2)

def on_error(ws, error):

    print(error)

def on_close(ws, close_status_code, close_msg):

    print("Connection Closed", close_status_code, close_msg)


def on_message(ws, message):

    data = json.loads(message)
    process_book_ticker(data)
    check_triangular_arbitrage_opportunity(triangles)

    if len(df) > 10:
        ws.close()


def process_book_ticker(data):
    
    symbol = data["s"]
    bid_price = Decimal(data["b"])
    ask_price = Decimal(data["a"])

    # Update or insert the data in the DataFrame
    update_dataframe(symbol, bid_price, ask_price)

def update_dataframe(symbol, bid_price, ask_price):
   
    global book_ticker_df

    # Check if the symbol is already in the DataFrame
    if symbol in book_ticker_df["Symbol"].values:
        # Update existing row
        book_ticker_df.loc[book_ticker_df["Symbol"] == symbol, ["Bid Price", "Ask Price"]] = [bid_price, ask_price]
    else:
        # Insert a new row
        new_row = pd.DataFrame([[symbol, bid_price, ask_price]], columns=book_ticker_df.columns)
        book_ticker_df = pd.concat([book_ticker_df, new_row], ignore_index=True)

def subscribe_to_book_tickers(pairs):
    # Construct the WebSocket URL for book tickers
    symbols = [f"{pair.lower()}@bookTicker" for pair in sample["pair"]]
    url = f"{BINANCE_WS_BASE_URL}{'/'.join(symbols)}"

    # Start the WebSocket connection
    ws = websocket.WebSocketApp(url, on_message=on_message, on_error=on_error, on_close=on_close)
    ws.run_forever()
    ws.close()





In [67]:
# Subscribe to book tickers for the specified pairs
subscribe_to_book_tickers(sample)

Connection Closed None None


In [68]:
book_ticker_df.head()

,Symbol,Bid Price,Ask Price
0,MATICUSDT,0.75690000,0.75700000
1,ETHUSDT,2348.72000000,2348.73000000
2,BTCUSDT,40274.52000000,40274.53000000
3,LTCBTC,0.00168900,0.00169000
4,TRXUSDT,0.10954000,0.10955000


In [69]:
df.head()

,Highest Arbitrage,A,B,C,Action
0,1.0016304111272505729,BTCUSDC,USDCUSDT,BTCUSDT,Buy
1,1.0016306600017243774,BTCUSDC,USDCUSDT,BTCUSDT,Buy
2,1.0016353886402250945,BTCUSDC,USDCUSDT,BTCUSDT,Buy
3,1.0016906423566245900,BTCUSDC,USDCUSDT,BTCUSDT,Buy
4,1.0016787043782924163,BTCUSDC,USDCUSDT,BTCUSDT,Buy
